In [1]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']

#  this doc would be on monthly & customer level
#  however the opportunity might not applicable for this.
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n\n#  this doc would be on monthly & customer level\n#  however the opportunity might not applicable for this.\n"

In [28]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
df_bill_type = pd.read_csv('./data_down/' + 'dim_dsc_billing_type_info.csv', sep = '\001') 
"""
pic part data
"""
sap_fina = pd.read_csv('./data_down/' + 'smart_hr_pl_0l.csv', sep = '\001') 
levin_table = pd.read_excel('./data_down/' + 'OU_Segementation2.xlsx', sheet_name = 2) 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa, df_bill,df_cost,df_esti,rel, df_bcaw, sap_fina]:
    colname_modi(i)
 
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

In [ ]:
# pd.set_option("display.max_rows", 15, "display.max_columns", 15)
# df_oppa[df_oppa['milestone'] == 'Closed-Lost']

# # loss_reason situation_update_next_steps description territory contract_value situation_update_next_steps sector product1
# # opportunity_type expected_golive_date	 contract_term contract_end_date

In [33]:
def data_prepare():
    """
    data prepare opportunity .
    """
    # df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    # df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df = df_oppa.copy()
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    return contract_summary

opportunity = data_prepare();opportunity.head(3)
# del contract_summary

,account_name_en,contract_end_date,account_id,bg,contract_term,milestone,opportunity_record,product1,product2,sector,annual_average_gross_profit_base,annual_average_gross_profit,annual_average_revenue_base,annual_average_revenue,contract_value,contract_end_yr
0,ABC,2021-06-28,e0209d33-e3aa-ea11-a812-000d3a851174,TSC,12.0,Closed-Canceled,New Business,Warehousing,Transportation,Technology,5000.0,5000.0,50000.0,50000.0,50000.0,2021
1,ADIDAS,2018-09-29,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,36.0,Contract Signed,New Business,Warehousing,Warehousing,Retail,950.4,1188.0,10777.6,13472.0,40416.0,2018
2,ADIDAS,2019-10-30,f64a8fa5-a304-4c85-9641-7205e77e70d8,HK,12.0,Contract Signed,Straightforward Renewals,Warehousing,Transportation,Retail,1087.2,1359.0,13123.2,16404.0,0.0,2019


In [ ]:
# df_list = [opportunity,df_bill,df_cost,df_esti]
# def check_ouname(df):
#     print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

# for i in df_list:
#     check_ouname(i)
# del df_list,i

In [ ]:
"""
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    """
    where u can set for month/year level.
    """
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4) 
    df = df.groupby(['account_name_en','account_name_cn', substr_year]).agg({
        **kwargs
    })
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    merge all to cust level \
        to_cust_level(df,  substr_year, **kwargs):
    """
    try:
        df1 = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

---
# billing

In [5]:
"""
~billing~
bill detail table now having 50+ bms_types. which is unacceptalble/
"""
def data_load_bms():
    df = df_bill.copy()
    df['month'] = df['bms_bill_end_date'].astype(str).str.slice(0,6) 
    df = df.merge(df_bill_type, left_on = 'bms_fee_type', right_on = 'billing_fee_type_code', how = 'left')
    bms = df.pivot_table(index= ['cost_center', 'month'],\
        columns= 'category',
        values='bms_amount',).reset_index().fillna(0)
    return bms
bms = data_load_bms()
bms.head(2)

category,cost_center,month,人力,场地,增值,报关代理,操作,短驳运输,索赔及返利,设备使用
0,ABBOTBJXWS,202003,8153.490566,94013.547170,1368.516509,0.0,18795.386792,0.0,0.0,16845.283019
1,ABBOTBJXWS,202005,16216.742138,182958.566038,1066.343553,0.0,55997.735849,0.0,0.0,17986.415094


In [6]:
bms_cus_yr = to_cust_level(df = bms, substr_year = 'month' ,\
    cost_center = set, 
    人力 = 'sum',
    场地 = 'sum',
    增值 = 'sum',
    报关代理 = 'sum',
    操作 = 'sum',
    短驳运输 = 'sum',
    索赔及返利 = 'sum',
    设备使用 = 'sum' ).reset_index()
bms_cus_yr.head(2)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_8544/4278657550.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


,account_name_en,account_name_cn,month,cost_center,人力,场地,增值,报关代理,操作,短驳运输,索赔及返利,设备使用
0,ASML,阿斯麦,2020,"{ASMLXGZGQS, ASMLXBJKCS, ASMLXSHZCS, ASMLXXAEP...",1.327222e+06,2.650783e+06,277290.273019,8.301644e+05,351038.906930,3.930000e+04,0.0,218274.787736
1,ASML,阿斯麦,2021,"{ASMLXGZGQS, ASMLXBJKCS, ASMLXSHZCS, ASMLXXAEP...",2.027038e+06,3.116845e+06,614587.514906,2.113024e+06,180360.231132,1.676826e+06,0.0,274845.181649


---
# cost

In [ ]:
# def tst(substr_year,**kwargs):
#     df_bill[substr_year] = df_bill[substr_year].astype(str).str.slice(0,4)
#     return  df_bill.groupby([substr_year, 'cost_center']).agg({
#         **kwargs
#     }).reset_index()

# tst(substr_year = 'bms_bill_end_date' ,\
#     bms_amount = 'sum', bms_fee_type = [set,'nunique'], bms_fee_name = set, bms_bill_id = 'nunique')

# test = df_bill.head(39)
# test.pivot_table(index= ['bms_bill_id', 'cost_center'],\
#     columns= ['bms_fee_type'],
#     values='bms_amount',)

In [10]:
"""
# cost
"""
def data_load_cost():
    df = df_cost
    df = df.dropna(axis=1, how = 'all').fillna(0)
    frame = pd.DataFrame(df.sum(axis = 0) == 0)
    df = df.loc[:, frame[frame[0] == False].index.tolist()]
    df['year'] = df['month'].astype(str).str.slice(0,4)
    return df


df_cost = data_load_cost()
df_cost.head(2)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_8544/3348520740.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  frame = pd.DataFrame(df.sum(axis = 0) == 0)


,id,ou_code,level,month,rev,gp,hr_cost,staff_cost,staff_cost_dir,hour_dir,...,staff_cost_bizos_actual,staff_roi_nmom,staff_cost_cm_mom,staff_cost_cm_nmom,rev_target,staff_cost_target,gp_target,hc_target,src_inc_day,year
0,1907,ABBOTBJXWS,site,201912,150.0,-66.0,53745.36,59.0,59.0,1144.0,...,357.72,4.85,3.77,-6.52,0.00,0.00,0.00,0.0,20211111,2019
1,10047,ABBOTBJXWS,site,202102,119.0,-45.0,52033.54,54.0,0.0,0.0,...,514.18,-3.38,-6.43,-0.90,141.53,53.33,-44.43,0.0,20211111,2021


In [12]:
cost_cus_yr = to_cust_level(df_cost, 'month',\
     rev = 'sum',
     gp = 'sum',
     hr_cost  = 'sum', 
     hc  = 'sum',
     hc_os  = 'sum',
     total_working_hrs  = 'sum',
     staff_cost_fixed  = 'sum',
     staff_cost_vari  = 'sum',
     staff_cost_os_fixed  = 'sum',
     staff_cost_os_vari  = 'sum',
     staff_cost_bizos_actual  = 'sum'
     ).reset_index()
cost_cus_yr.head()

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_8544/4278657550.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[substr_year] = df[substr_year].astype(str).str.slice(0,4)


,account_name_en,account_name_cn,month,rev,gp,hr_cost,hc,hc_os,total_working_hrs,staff_cost_fixed,staff_cost_vari,staff_cost_os_fixed,staff_cost_os_vari,staff_cost_bizos_actual
0,APPLE,苹果,2019,126056.0,26870.0,11437374.08,623.20,240.0,19044.0,7676743.92,1619876.84,1224772.86,494148.46,421832.00
1,APPLE,苹果,2020,95826.0,16104.0,8121430.38,382.40,292.0,58010.0,4652451.54,1102658.84,1438071.98,701433.04,226814.98
2,APPLE,苹果,2021,75900.0,10902.0,6518272.60,313.48,246.0,0.0,3900571.58,729040.64,1313336.66,313117.56,262206.16
3,Abbott Nutrition,雅培 营养品,2019,25973.0,-3558.0,5798148.25,250.41,206.0,8690.0,3429201.52,525409.46,1013933.18,355173.48,474430.61
4,Abbott Nutrition,雅培 营养品,2020,23710.0,-3875.0,5654603.72,282.26,170.0,32832.0,3625931.43,474042.29,826245.84,252088.07,476296.09


---
# merge

In [18]:
bms_cus_yr['account_name_en'].nunique()

116

In [15]:
test = bms_cus_yr.merge(cost_cus_yr, on = ['account_name_en', 'month'], how = 'inner')

In [16]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
test.account_name_en.nunique()

40

In [ ]:
test['account_name_en'].nunique()

In [ ]:
# test[test['account_name_en'].str.lower().str.match('bose')]

In [ ]:
# df_oppa[df_oppa['account_name_en'].str.lower().str.match('bmw').fillna(False)][['bg','opportunity_number', 'opportunity_name', 'account_id',
#        'account_name_en', 'account_name_cn', 'actual_close_date',
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue',
#        'bca_ref_number', 'bca_status']]

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

